# Proyecto de predicción de retrasos en vuelos comerciales

- El objetivo de este caso de estudio es proponer un enfoque para encontrar causales y estrategias para reducción o prevención en pérdidas por retrasos en los sistemas de transporte aéreo.

- Para lograr este objetivo, debemos definir el foco desde donde se hará el análisis, así como las herramientas mas adhoc para el análisis de la información.

- Se nos es dado un dataset para trabajar, por lo que tenemos algunas preguntase que quiero proponer a responder para ayudarnos a identificar el enfoque del estudio:

    Preguntas:	Cómo podemos usar esta información para identificar causas de retrasos?
                    Cuántas causas podemos identificar a partir de esta información?
                    Qué otra información podríamos integrar? deberíamos integarar más?
                    Qué enfoque se debe tomar? Delay propagation, root delay or cancellation
                    
- La información con la que contamos son 3 archivos con información de las aerolineas, aeropuertos e información de vuelos:

        Airline.csv	  =>  IATA-CODE, AIRLNE
        Airports.csv  =>  IATA_CODE, AIRPORT, CITY, STATE, COUNTRY, LATITUDE, LONGITUDEt
        Flights.csv   =>  YEAR, MONTH, DAY, DAY_OF_WEEK, AIRLINE, FLIGHT_NUMBER, TAIL_NUMBER, ORIGIN_ARIPORT,
							DESTINATION_AIRPORT, SCHEDULED_DEPARTURE, WHEELS_OFF, TAXI_OUT, DEPARTURE_TIME,
							SCHEDULED_TIME, AIR_TIME, DISTANCE, TAXI_IN, ELAPSED_TIME, SCHEDULED_ARRIVAL,
							ARRIVAL_TIME, ARRIVAL_DELAY, DIVERTED, CANCELLED, CANCELLATION_REASON,
                            AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLNE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY
                                        
- Al hacer una exploración inicial, me gustaría proponer tambien una hipótesis de las correlaciones que pudiéramos hacer de la información que tenemos:

        Hipótesis:	1. Con esta informacion podemos encontrar correlaciones entre retrasos por aerolinea
                    lo que nos podría indicar un problema en las operaciones de ciertas aerolineas
                    2. Podemos observar si hay correlaciones entre los aeropuertos de salida y destinos.
                    Nos podría indicar si hay rutas que son más problemáticas; lo que podría ser
                    consecuencia de condiciones ambientales, rutas en esas ciudades (para que llegen
                    los pasajeros al aeropuerto), ciudades más afluentes (eventos sociales), trabajos de
                    construcción en carreteras
                    3. Tiempos de retraso por aeropuerto (similar a 1) relacionado a las operaciones
                    aeroportuarias (manejo de pista, control aereo)
                    4. Razones de cancelación: buscar patrones que relacionen causas con aeropuertos, 
                    rutas, aviones, días específicos
                    5. Razones de cancelación: cancelaciones más frecuentes en ciertas épocas del año
                    o correlación con 2
                    6. En los casos de desvíos, a qué aeropuertos se reenrutan las aeronaves y desde qué
                    areopuertos (se puede mejorar esto?)

## Exploración
- Para hacer una exploración inicial de la información, usaremos algunos módulos de python para importar los datasets y hacer algunas gráficas

In [2]:
import pandas as pd

airline_path = 'datasets/airlines.csv'
airport_path = 'datasets/airports.csv'
flight_path = 'datasets/flights.csv'

airline_df = pd.read_csv(airline_path)
airport_df = pd.read_csv(airport_path)
flight_df = pd.read_csv(flight_path)

C:\Users\i0nat\AppData\Local\Temp\ipykernel_13848\3595243444.py:9: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flight_df = pd.read_csv(flight_path)


## Airline dataset

In [3]:
airline_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [4]:
airline_df.dtypes

IATA_CODE    object
AIRLINE      object
dtype: object

In [5]:
airline_df.describe()

,IATA_CODE,AIRLINE
count,14,14
unique,14,14
top,UA,United Air Lines Inc.
freq,1,1


## Airport dataset

In [6]:
airport_df

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [7]:
airport_df.dtypes

IATA_CODE     object
AIRPORT       object
CITY          object
STATE         object
COUNTRY       object
LATITUDE     float64
LONGITUDE    float64
dtype: object

In [8]:
len(airport_df['CITY'].unique())

308

## Flights dataset

In [16]:
flight_df.iloc[0]

YEAR                     2015
MONTH                       1
DAY                         1
DAY_OF_WEEK                 4
AIRLINE                    AS
FLIGHT_NUMBER              98
TAIL_NUMBER            N407AS
ORIGIN_AIRPORT            ANC
DESTINATION_AIRPORT       SEA
SCHEDULED_DEPARTURE         5
DEPARTURE_TIME         2354.0
DEPARTURE_DELAY         -11.0
TAXI_OUT                 21.0
WHEELS_OFF               15.0
SCHEDULED_TIME          205.0
ELAPSED_TIME            194.0
AIR_TIME                169.0
DISTANCE                 1448
WHEELS_ON               404.0
TAXI_IN                   4.0
SCHEDULED_ARRIVAL         430
ARRIVAL_TIME            408.0
ARRIVAL_DELAY           -22.0
DIVERTED                    0
CANCELLED                   0
CANCELLATION_REASON       NaN
AIR_SYSTEM_DELAY          NaN
SECURITY_DELAY            NaN
AIRLINE_DELAY             NaN
LATE_AIRCRAFT_DELAY       NaN
WEATHER_DELAY             NaN
Name: 0, dtype: object

In [14]:
flight_df.value_counts()

Series([], dtype: int64)

In [15]:
flight_df.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,...,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.335204e+03,9.370158e+00,1.607166e+01,1.357171e+03,...,1.493808e+03,1.476491e+03,4.407057e+00,2.609863e-03,1.544643e-02,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964233e+02,3.708094e+01,8.895574e+00,4.980094e+02,...,5.071647e+02,5.263197e+02,3.927130e+01,5.102012e-02,1.233201e-01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,4.000000e+00,8.000000e+00,2.000000e+00,7.300000e+02,9.170000e+02,9.210000e+02,-5.000000e+00,1.100000e+01,9.350000e+02,...,1.110000e+03,1.059000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,7.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,1.330000e+03,-2.000000e+00,1.400000e+01,1.343000e+03,...,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.230000e+03,1.730000e+03,1.740000e+03,7.000000e+00,1.900000e+01,1.754000e+03,...,1.918000e+03,1.917000e+03,8.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,1.900000e+01,2.900000e+01,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.400000e+03,1.988000e+03,2.250000e+02,2.400000e+03,...,2.400000e+03,2.400000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


In [13]:
flight_df['CANCELLATION_REASON'].unique()

array([nan, 'A', 'B', 'C', 'D'], dtype=object)

In [6]:
flight_df.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [31]:
len(flight_df['FLIGHT_NUMBER'].unique())

6952

## Inspección de los datos

- Una propuesta de inspección incial como se mencionó en la planteación de hipótesis, es comenzar a separa los parámetros que nos pueden ayudar a identificar ciertas correlaciones para diferentes segmentos

- Por ejemplo, podemos comenzar a ver la distribución de los parámetros que estána relacionados con las operaciones aeropuertarias, con el objetivo de encontrar patrones para ciertos aeropuertos(?), ciudades(?), épocas del año(?)

In [23]:
airport_time_params = ['DEPARTURE_DELAY', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_OUT', 'TAXI_IN', 'DEPARTURE_TIME', 'ARRIVAL_DELAY']

In [25]:
flight_df[airport_time_params].describe()

,DEPARTURE_DELAY,WHEELS_OFF,WHEELS_ON,TAXI_OUT,TAXI_IN,DEPARTURE_TIME,ARRIVAL_DELAY
count,5.732926e+06,5.730032e+06,5.726566e+06,5.730032e+06,5.726566e+06,5.732926e+06,5.714008e+06
mean,9.370158e+00,1.357171e+03,1.471469e+03,1.607166e+01,7.434971e+00,1.335204e+03,4.407057e+00
std,3.708094e+01,4.980094e+02,5.221879e+02,8.895574e+00,5.638548e+00,4.964233e+02,3.927130e+01
min,-8.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.700000e+01
25%,-5.000000e+00,9.350000e+02,1.054000e+03,1.100000e+01,4.000000e+00,9.210000e+02,-1.300000e+01
50%,-2.000000e+00,1.343000e+03,1.509000e+03,1.400000e+01,6.000000e+00,1.330000e+03,-5.000000e+00
75%,7.000000e+00,1.754000e+03,1.911000e+03,1.900000e+01,9.000000e+00,1.740000e+03,8.000000e+00
max,1.988000e+03,2.400000e+03,2.400000e+03,2.250000e+02,2.480000e+02,2.400000e+03,1.971000e+03


- Este grupo de datos por sí solo no nos dice mucho, pero si agrupamos con el número de vuelo para identificar la ruta (aeropuerto de origen y destino), podemos buscar distsribuciones de compotramientos por ruta por ejemplo.

In [34]:
flight_df.groupby("FLIGHT_NUMBER")[airport_time_params].describe()

DEPARTURE_DELAY                                                \
                        count       mean        std   min   25%   50%   75%   
FLIGHT_NUMBER                                                                 
1                      2366.0   2.410820  21.522442 -19.0  -5.0  -3.0   1.0   
2                      1962.0   5.830785  29.349620 -16.0  -5.0  -2.0   4.0   
3                      2857.0   9.346867  35.926577 -24.0  -5.0  -2.0   8.0   
4                      1755.0   4.208547  24.921538 -16.0  -5.0  -2.0   2.0   
5                      2247.0   8.120605  47.771310 -22.0  -5.0  -2.0   5.0   
...                       ...        ...        ...   ...   ...   ...   ...   
8445                      1.0 -11.000000        NaN -11.0 -11.0 -11.0 -11.0   
9320                      1.0  19.000000        NaN  19.0  19.0  19.0  19.0   
9793                      0.0        NaN        NaN   NaN   NaN   NaN   NaN   
9794                      1.0  22.000000        NaN  22.0  22.0  22.0  22.0   
9855                      0.0        NaN        NaN   NaN   NaN   NaN   NaN   

                      WHEELS_OFF               ... DEPARTURE_TIME          \
                  max      count         mean  ...            75%     max   
FLIGHT_NUMBER                                  ...                          
1               298.0     2365.0   881.147146  ...          859.0  2009.0   
2               368.0     1961.0  1322.745028  ...         1654.0  2128.0   
3               665.0     2855.0  1243.172329  ...         1651.0  2356.0   
4               279.0     1755.0  1260.849573  ...         1316.5  2359.0   
5              1264.0     2246.0  1040.399822  ...         1211.5  2400.0   
...               ...        ...          ...  ...            ...     ...   
8445            -11.0        1.0  1550.000000  ...         1539.0  1539.0   
9320             19.0        1.0   748.000000  ...          719.0   719.0   
9793              NaN        0.0          NaN  ...            NaN     NaN   
9794             22.0        1.0  1259.000000  ...         1237.0  1237.0   
9855              NaN        0.0          NaN  ...            NaN     NaN   

              ARRIVAL_DELAY                                                \
                      count       mean        std   min   25%   50%   75%   
FLIGHT_NUMBER                                                               
1                    2360.0  -1.755932  27.491364 -73.0 -15.0  -6.0   6.0   
2                    1961.0  -2.864865  33.219010 -68.0 -19.0  -9.0   4.0   
3                    2848.0   3.027739  38.785120 -73.0 -15.0  -4.0   9.0   
4                    1754.0  -4.733181  28.450687 -57.0 -20.0  -9.0   2.0   
5                    2242.0   3.387154  51.687127 -71.0 -15.0  -6.0   7.0   
...                     ...        ...        ...   ...   ...   ...   ...   
8445                    1.0 -13.000000        NaN -13.0 -13.0 -13.0 -13.0   
9320                    1.0  36.000000        NaN  36.0  36.0  36.0  36.0   
9793                    0.0        NaN        NaN   NaN   NaN   NaN   NaN   
9794                    0.0        NaN        NaN   NaN   NaN   NaN   NaN   
9855                    0.0        NaN        NaN   NaN   NaN   NaN   NaN   

                       
                  max  
FLIGHT_NUMBER          
1               298.0  
2               330.0  
3               659.0  
4               267.0  
5              1295.0  
...               ...  
8445            -13.0  
9320             36.0  
9793              NaN  
9794              NaN  
9855              NaN  

[6952 rows x 56 columns]